In [171]:
# data structure and working with data api's
import numpy as np
import pandas as pd
# graphing/plotting super cool charts api
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# trade api
import alpaca_trade_api as trader
# stock data api
import yfinance as yf
# processing data/getting indicators api
import talib
# timing stuff
import time
import datetime
from pandas.tseries.offsets import BDay

In [121]:
class Indicators():
    def __init__(self, mode='Close'):
        
        # initialization function, assigns the variable indicators all the indicators so that default the optimizer can use that as its primary means of attack.
        # the mode is just whether we will base stuff on the Close, Open, High or Low
        
        self.indicators = ['SMA20', 'SMA60', 'SMA50', 'SMA/EMA20', 'SMA/EMA50', 'EMA12', 'EMA50',
                            'BBS', 'RSI', 'TEMA', 'MACD1', 'MACD2', 'MACD3', 'HTITRENDLINE', 'PSAR']
        self.Mode = mode
        
    def indicator(self, df, indicator, plot=False):
        # necessary method to use when working with data and needing to get an indicator. Needed for trailing stops as well, which
        # themselves are the crutch of the scoring.
        
        if indicator == 'SMA20':
            return self.Averages(df, plot=plot)
        elif indicator == 'SMA60':
            return self.Averages(df, period1=60, period2=90, plot=plot)
        elif indicator == 'SMA50':
            return self.Averages(df, period1=50, period2=200, plot=plot)
        elif indicator == 'SMA/EMA20':
            return self.Averages(df, period1=20, period2=20, mode=2, plot=plot)
        elif indicator == 'SMA/EMA50':
            return self.Averages(df, period1=50, period2=50, mode=2, plot=plot)
        elif indicator == 'EMA12':
            return self.Averages(df, period1=12, period2=26, mode=3, plot=plot)
        elif indicator == 'EMA50':
            return self.Averages(df, period1=50, period2=50, mode=3, plot=plot)
        elif indicator == 'TEMA':
            return self.Averages(df, period1=25, period2=50, mode=4, plot=plot)
        elif indicator == 'KAMA':
            return self.Averages(df, period1=25, period2=50, mode=5, plot=plot)
        elif indicator == 'BBS':
            return self.Bollingers(df, plot=plot)
        elif indicator == 'RSI':
            return self.RSI(df, plot=plot)
        elif indicator == 'MACD1':
            return self.MACD(df, mode=1, plot=plot)
        elif indicator == 'MACD2':
            return self.MACD(df, mode=2, plot=plot)
        elif indicator == 'MACD3':
            return self.MACD(df, mode=3, plot=plot)
        elif indicator == 'HTITRENDLINE':
            return self.HilbertTransform(df, plot=plot)
        elif indicator == 'PSAR':
            return self.PSAR(df, acc=0.075, maxi=0.1, plot=plot)
        return None
    
    # Following are the trailing stops, going to be necessary to determine when to exit the strats.
    # Namely the Average True Range Trailing Stops are the needed operation for the entire code (but all code is neccesary for all the code ig)
    
    def TrailStop(self, df, indicator, increment, buy_order=True):
        buy, sell, dummy, dummy = self.indicator(df, indicator)
        bought = False
        TrailingPrice = pd.Series(name = 'TrailingPrice', dtype='float')
        Exit = pd.Series(name = 'Exit', dtype='int')
        i = 0
        if buy_order:
            df = pd.concat([df, buy.astype(bool).astype(int).rename('Entry')], axis=1)
            df.Entry.replace(np.nan, 0, inplace=True)
            for k, index in enumerate(df.index):
                if bought:
                    if df['Close'][index] - (TrailingPrice[TrailingPrice.index[i-1]] + increment) > 0:
                        TrailingPrice[index] = df['Close'][index] - increment
                    else:
                        TrailingPrice[index] = TrailingPrice[TrailingPrice.index[i-1]]
                    i += 1
                    if df['Close'][index] < TrailingPrice[index]:
                        bought = False
                        Exit[index] = 1
                elif df['Entry'][index].astype(bool):
                    TrailingPrice[index] = df['Close'][index] - increment
                    bought = True
                    i += 1
        else:
            df = pd.concat([df, sell.astype(bool).astype(int).rename('Entry')], axis=1)
            df.Entry.replace(np.nan, 0, inplace=True)
            for k, index in enumerate(df.index):
                if bought:
                    if (df['Close'][index] + increment) < TrailingPrice[TrailingPrice.index[i-1]]:
                        TrailingPrice[index] = df['Close'][index] + increment
                    else:
                        TrailingPrice[index] = TrailingPrice[TrailingPrice.index[i-1]]
                    i += 1
                    if df['Close'][index] > TrailingPrice[index]:
                        bought = False
                        Exit[index] = 1
                elif df['Entry'][index].astype(bool):
                    TrailingPrice[index] = df['Close'][index] + increment
                    bought = True
                    i += 1
        df = pd.concat([df, TrailingPrice, Exit], axis=1)
        return df.Entry.replace(0, np.nan).dropna(), df.Exit.dropna()
        
    def ATR_TR(self, df, indicator, mult=3, buy_order=True):
        buy, sell, dummy, dummy = self.indicator(df, indicator, plot=False)
        atr = self.ATR(df, plot=False)
        bought = False
        TrailingPrice = pd.Series(name = 'TrailingPrice', dtype='float')
        Exit = pd.Series(name = 'Exit', dtype='int')
        i = 0
        if buy_order:
            df = pd.concat([df, buy.astype(bool).astype(int).rename('Entry')], axis=1)
            df.Entry.replace(np.nan, 0, inplace=True)
            for k, index in enumerate(df.index):
                increment = atr[index] * mult
                if bought:
                    if df['Close'][index] - (TrailingPrice[TrailingPrice.index[i-1]] + increment) > 0:
                        TrailingPrice[index] = df['Close'][index] - increment
                    else:
                        TrailingPrice[index] = TrailingPrice[TrailingPrice.index[i-1]]
                    i += 1
                    if df['Close'][index] < TrailingPrice[index]:
                        bought = False
                        Exit[index] = 1
                elif df['Entry'][index].astype(bool):
                    TrailingPrice[index] = df['Close'][index] - increment
                    bought = True
                    i += 1
        else:
            df = pd.concat([df, sell.astype(bool).astype(int).rename('Entry')], axis=1)
            df.Entry.replace(np.nan, 0, inplace=True)
            for k, index in enumerate(df.index):
                increment = atr[index] * mult
                if bought:
                    if (df['Close'][index] + increment) < TrailingPrice[TrailingPrice.index[i-1]]:
                        TrailingPrice[index] = df['Close'][index] + increment
                    else:
                        TrailingPrice[index] = TrailingPrice[TrailingPrice.index[i-1]]
                    i += 1
                    if df['Close'][index] > TrailingPrice[index]:
                        bought = False
                        Exit[index] = 1
                elif df['Entry'][index].astype(bool):
                    TrailingPrice[index] = df['Close'][index] + increment
                    bought = True
                    i += 1
        df = pd.concat([df, TrailingPrice, Exit], axis=1)
        return df.Entry.replace(0, np.nan).dropna(), df.Exit.dropna(), TrailingPrice
    
    # Following are the actual indicators, here's where a lot of the action happens.
    # The indicators look like a lot of things are going on, but stripped away of the plotting necessities they
    # would be extremely short, and that will happen as soon as the actual paper trading stuffs is completed.
        
    def ATR(self, df, period=14, plot=True):
        atr = talib.ATR(df.High, df.Low, df.Close, timeperiod=period)
        if plot:
            fig = make_subplots(rows=2, cols=1, column_widths=[1], row_heights=[0.8, 0.2], specs =
                               [ [ {'type': 'candlestick'} ], [ {'type': 'scatter'} ] ], shared_xaxes=True )
            trace1 = go.Candlestick(x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close, name='Stock Data')
            trace2 = go.Scatter(x=atr.index, y=atr, name='Average True Range')
            fig.add_trace(trace1, row=1, col=1)
            fig.add_trace(trace2, row=2, col=1)
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        return atr
    
    def Averages(self, df, period1=20, period2=50, mode=1, plot=True):
        # ok so looks very complicated, but I will try to simplify for myself:
        # close gets the Close data for the specified ticker.
        # From there, we get two SMAs that default to 20-period and 50-period, but you can specify if you want.
        # Taking the difference of the two SMAs can tell us when to buy and sell -
        # we call get_buy_sell, which iterates through the differences, and when the 20-p goes above 50-p it's a buy signal,
        # and vice versa when 20-p goes below 50-p.
        # Plotting is interesting: we make multiple traces with different plots, and then overlay the plots, leading to complete graph that is also interactive.
        close = df[self.Mode]
        if mode == 1:
            avg1 = talib.SMA(close, timeperiod=period1)
            avg2 = talib.SMA(close, timeperiod=period2)
        elif mode == 2:
            avg1 = talib.SMA(close, timeperiod=period1)
            avg2 = talib.EMA(close, timeperiod=period2)
        elif mode == 3:
            avg1 = talib.EMA(close, timeperiod=period1)
            avg2 = talib.EMA(close, timeperiod=period2)
        elif mode == 4:
            avg1 = talib.TEMA(close, timeperiod=period1)
            avg2 = talib.TEMA(close, timeperiod=period2)
        elif mode == 5:
            avg1 = talib.KAMA(close, timeperiod=period1)
            avg2 = talib.KAMA(close, timeperiod=period2)
        diff = avg1 - avg2
        buy_times, sell_times = self.get_buy_sell(diff)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
            trace1, trace4, trace5 = self.get_traces(df, buy_times, sell_times)
            trace2 = go.Scatter(x=avg1.index, y=avg1, name=str(period1)+' Period Average')
            trace3 = go.Scatter(x=avg2.index, y=avg2, name=str(period2)+' Period Average')
            fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, close[avg1 > avg2].rename('Good'), close[avg1 <= avg2].rename('Bad')
    
    def Bollingers(self, df, period=20, plot=True):
        close = df[self.Mode]
        upper, middle, lower = talib.BBANDS(close, timeperiod=period)
        up_mid, middle, low_mid = talib.BBANDS(close, timeperiod=period, nbdevup=1, nbdevdn=1)
        diff1 = close - up_mid
        diff2 = close - low_mid
        buy_times, sell_times = self.get_buy_sell(diff1, diff2)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
            trace1, trace6, trace7 = self.get_traces(df, buy_times, sell_times)
            trace2 = go.Scatter(x=middle.index, y=upper, line={'color':'rgba(0, 255, 0, 0.8)'}, name='Buy Range', showlegend=False)
            trace3 = go.Scatter(x=middle.index, y=up_mid, fill='tonexty', fillcolor='rgba(0, 255, 0, 0.2)', line={'color':'rgba(255, 255, 0, 0.8)'}, name='No Man\'s Land')
            trace4 = go.Scatter(x=middle.index, y=low_mid, fill='tonexty', fillcolor='rgba(255, 255, 0, 0.2)', line={'color':'rgba(255, 255, 0, 0.8)'}, name='No Man\'s Land')
            trace5 = go.Scatter(x=middle.index, y=lower, fill='tonexty', fillcolor='rgba(255, 0, 0, 0.2)', line={'color':'rgba(100, 0, 0, 0.8)'}, name='Sell Range')
            fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5, trace6, trace7])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, close[close > up_mid].rename('Good'), close[close <= low_mid].rename('Bad')
    
    def RSI(self, df, period=14, sell_at=None, buy_at=None, plot=True):
        close = df[self.Mode]
        rsi = talib.RSI(close, timeperiod=period)
        sell_point = rsi.max() - ( rsi.std() / 2 ) or sell_at
        buy_point = rsi.min() + ( rsi.std() / 2 ) or buy_at
        diff1 = rsi - sell_point
        diff2 = rsi - buy_point
        sell_times, buy_times = self.get_buy_sell(diff1, diff2)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
            fig = make_subplots(rows=2, cols=1, column_widths=[1], row_heights=[0.8, 0.2], specs =
                               [ [ {'type': 'candlestick'} ], [ {'type': 'scatter'} ] ], shared_xaxes=True )
            trace1, trace2, trace3 = self.get_traces(df, buy_times, sell_times)
            trace4 = go.Scatter(x=rsi.index, y=rsi, name='RSI')
            fig.add_trace(trace1, row=1, col=1)
            fig.add_trace(trace2, row=1, col=1)
            fig.add_trace(trace3, row=1, col=1)
            fig.add_trace(trace4, row=2, col=1)
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, close[rsi >= buy_point], close[rsi <= sell_point]
    
    def TripExAverages(self, df, period1=25, period2=50, plot=True):
        close = df[self.Mode]
        tea1 = talib.TEMA(close, timeperiod=period1)
        tea2 = talib.TEMA(close, timeperiod=period2)
        diff = tea1 - tea2
        buy_times, sell_times = self.get_buy_sell(diff)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
            trace1, trace4, trace5 = self.get_traces(df, buy_times, sell_times)
            trace2 = go.Scatter(x=tea1.index, y=tea1, name=str(period1)+' Period TEMA')
            trace3 = go.Scatter(x=tea2.index, y=tea2, name=str(period2)+' Period TEMA')
            fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, close[tea1 > tea2].rename('Good'), close[tea1 <= tea2].rename('Bad')
    
    def MACD(self, df, period1=12, period2=26, period3=9, mode=1, plot=True):
        close = df[self.Mode]
        macd, macd_avg, macd_hist = talib.MACD(close, fastperiod=period1, slowperiod=period2, signalperiod=period3)
        if mode == 1:
            diff = macd.copy()
        elif mode == 2:
            diff = macd_hist.copy()
        else:
            diff = macd_avg - macd
        buy_times, sell_times = self.get_buy_sell(diff)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
            fig = make_subplots(rows=2, cols=1, column_widths=[1], row_heights=[1, 1], specs =
                               [ [ {'type': 'candlestick'} ], [ {'type': 'bar'} ] ], shared_xaxes=True )
            trace1, trace2, trace3 = self.get_traces(df, buy_times, sell_times)
            trace4 = go.Bar(x=macd_hist[macd_hist > 0].index, y=macd_hist[macd_hist > 0], marker_color='green', name='MACD Histogram')
            trace5 = go.Bar(x=macd_hist[macd_hist <= 0].index, y=macd_hist[macd_hist <= 0], marker_color='red', name='MACD Histogram')
            trace6 = go.Scatter(x=macd.index, y=macd, name='MACD')
            trace7 = go.Scatter(x=macd_avg.index, y=macd_avg, name='MACD Average')
            fig.add_trace(trace1, row=1, col=1)
            fig.add_trace(trace2, row=1, col=1)
            fig.add_trace(trace3, row=1, col=1)
            fig.add_trace(trace4, row=2, col=1)
            fig.add_trace(trace5, row=2, col=1)
            fig.add_trace(trace6, row=2, col=1)
            fig.add_trace(trace7, row=2, col=1)
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, None, None
    
    def HilbertTransform(self, df, plot=True):
        close = df[self.Mode]
        htit = talib.HT_TRENDLINE(close)
        diff = close - htit
        buy_times, sell_times = self.get_buy_sell(diff)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
            trace1, trace3, trace4 = self.get_traces(df, buy_times, sell_times)
            trace2 = go.Scatter(x=htit.index, y=htit, name='Hilbert Transform Instantaneous Trendline')
            fig = go.Figure(data=[trace1, trace2, trace3, trace4])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, None, None
    
    def PSAR(self, df, acc=0.01, maxi=0.1, plot=True):
        close = df[self.Mode]
        sar = talib.SAR(df.High, df.Low, acceleration = acc, maximum=maxi)
        diff = close - sar
        buy_times, sell_times = self.get_buy_sell(diff)
        if plot:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close)
            trace1, trace3, trace4 = self.get_traces(df, buy_times, sell_times)
            trace2 = go.Scatter(x=sar.index, y=sar, name='Parabalic Stop and Reverse')
            fig = go.Figure(data=[trace1, trace2, trace3, trace4])
            fig.update_layout(xaxis_rangeslider_visible=False)
            fig.show()
        else:
            buy_times, sell_times = self.parse_buy_sell(buy_times, sell_times, close, clean=False)
        return buy_times, sell_times, None, None
    
    
    # Following are methods helping with the coding of indicators, they are very clutch
    
    def get_buy_sell(self, diff1, diff2=0):
        try:
            if diff2 == 0:
                diff2 = diff1
        except ValueError:
            pass
        buy_times = []
        sell_times = []
        for i in range(1, len(diff1)):
            if (diff1[i-1] <= 0 and diff1[i] >= 0):
                buy_times.append(diff1.index[i])
            elif (diff2[i-1] >= 0 and diff2[i] <= 0):
                sell_times.append(diff2.index[i])
        return buy_times, sell_times
    
    def parse_buy_sell(self, buy, sell, close, clean=True):
        df = pd.concat([close[buy].rename('Buy', inplace=True).to_frame(), close[sell].rename('Sell', inplace=True).to_frame()], axis=1)
        df.index.name = 'Date/Time'
        df.sort_values('Date/Time', inplace=True)
        if clean:
            current=''
            for index in df.index:
                if np.isnan(df.Buy[index]):
                    if current == 'Sell':
                        df.drop(labels=index, axis=0, inplace=True)
                    else:
                        current = 'Sell'
                else:
                    if current == 'Buy':
                        df.drop(labels=index, axis=0, inplace=True)
                    else:
                        current = 'Buy'
        buy = df['Buy']
        sell = df['Sell']
        buy.dropna(axis=0, inplace=True)
        sell.dropna(axis=0, inplace=True)
        return buy, sell
    
    def get_traces(self, df, buy_times, sell_times):
        trace1 = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Stock Data')
        trace2 = go.Scatter(x=buy_times.index, y=buy_times, mode='markers', name='Buy Points', marker_color='green', marker_size=10)
        trace3 = go.Scatter(x=sell_times.index, y=sell_times, mode='markers', name='Sell Points', marker_color='red', marker_size=10)
        return trace1, trace2, trace3
    
    # functions for self use in graphing and dependencies, unrelated to the evaluation pipeline
    
    def plot(self, df, buy_times='SMA20', sell_times='SMA20'):
        buy_times, dummy, dummy, dummy = self.indicator(df, buy_times)
        dummy, sell_times, dummy, dummy = self.indicator(df, sell_times)
        trace1, trace2, trace3 = self.get_traces(df, buy_times, sell_times)
        fig = go.Figure(data=[trace1, trace2, trace3])
        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.show()
    

In [323]:
class Stoinks:    
    def __init__(self, tickers=None,indicators=None, mode='Close', margin=1, run=1, history=('1d', '1m')):
        # when you make a new stoinks object, it'll automatically just do all this to initialize the object, 'Averages60', 'Averages50'
        # to see method explanations go to those methods, self.Tickers = tickers just is initializing that variable,
        # same with self.Indicators
        start = time.time()
        self.Process = Indicators(mode)
        if tickers is None:
            self.Tickers = ['msft', 'aapl', 'tsla', 'oxy']
        else:
            self.Tickers = tickers
        if indicators is None:
            self.Indicators = self.Process.indicators.copy()
        else:
            self.Indicators = indicators
        if run == 1:
            self.run(history, margin=margin)
        elif run == 2:
            self.run(history, optimize=False, margin=margin)
        end = time.time()
        print(str(end - start)+' Seconds Elasped')
    
    # the following methods are used as part of the optimization pipeline, which figures out the best indicator per stock,
    # and then based off that the best stocks themselves
    
    def run(self, history, optimize=True, margin=5):
        self.parse_data()
        self.histories(period=history[0], interval=history[1])
        if optimize:
            self.score_all()
            self.optimize(margin=margin)
        
    def parse_data(self):
        # fills up the dictionary stocks, which has the ticker objects that you access using the symbol,
        # eg: self.Stocks['aapl'] gives the ticker object for apple (ticker is the object type used by yfinance)
        self.Stocks = {}
        for symbol in self.Tickers:
            self.Stocks[symbol] = yf.Ticker(symbol)
        
    def histories(self, period='1d', interval='1m'):
        # fills up the dictionary object histories, which has dataframes that you access using the symbol,
        # eg: self.Histories['msft'] gives the microsoft stock dataframe for the specified period and interval during initialization
        self.Histories = {}
        i = 0
        while i < len(self.Tickers):
            try:
                self.Histories[self.Tickers[i]] = self.Stocks[self.Tickers[i]].history(period=period, interval=interval)
                if self.Histories[self.Tickers[i]].index[0] is None:
                    print('Achievement Unlocked: How did we get here?')
                i += 1
            except IndexError:
                print(i)
                print(self.Tickers[i])
                self.Histories.pop(self.Tickers[i])
                self.Stocks.pop(self.Tickers[i])
                self.Tickers.remove(self.Tickers[i])
                continue
    
    def optimize(self, margin=5):
        self.Optimal = {}
        self.Profitable = {}
        self.Buyable = {}
        self.Sellable = {}
        for ticker in self.Scores:
            self.Optimal[ticker] = {}
            buffer1 = self.Scores[ticker]['buy_indicator'][0]
            buffer2 = self.Scores[ticker]['sell_indicator'][0]
            self.Optimal[ticker]['buy_indicator'] = buffer1[0]
            self.Optimal[ticker]['sell_indicator'] = buffer2[0]
            if buffer1[1] >= margin and buffer2[1] >= margin:
                self.Profitable[ticker] = {}
                self.Profitable[ticker]['buy_indicator'] = buffer1[0]
                self.Profitable[ticker]['sell_indicator'] = buffer2[0]
            elif buffer1[1] >= margin and buffer2[1] > 0:
                self.Buyable[ticker] = {}
                self.Buyable[ticker]['buy_indicator'] = buffer1[0]
                self.Buyable[ticker]['sell_indicator'] = buffer2[0]
            elif buffer2[1] >= margin and buffer1[1] > 0:
                self.Sellable[ticker] = {}
                self.Sellable[ticker]['buy_indicator'] = buffer1[0]
                self.Sellable[ticker]['sell_indicator'] = buffer2[0]
        self.Optimal = pd.DataFrame(self.Optimal)
        
        x = lambda y: [[x, self.Scores[x]['buy_indicator'][0][0], self.Scores[x]['sell_indicator'][0][0]] for x in y]
        self.Goods = x({**self.Profitable, **self.Buyable, **self.Sellable})
        
        return self.Optimal, self.Profitable
    
    def score_all(self):
        self.Scores = {}
        for ticker in self.Tickers:
            wrapper = {'buy_indicator': [], 'sell_indicator': []}
            for indicator in self.Indicators:
                wrapper['buy_indicator'].append([indicator, 100*self.cost_score(ticker, indicator)[0]/self.Histories[ticker].Close.mean()])
                wrapper['sell_indicator'].append([indicator, 100*self.cost_score(ticker, indicator, buy_order=False)[0]/self.Histories[ticker].Close.mean()])
            wrapper['buy_indicator'].sort(key=lambda x:x[1], reverse=True)
            wrapper['sell_indicator'].sort(key=lambda x:x[1], reverse=True)
            self.Scores[ticker] = wrapper
        return self.Scores
        
    def cost_score(self, ticker, indicator, buy_order=True):
        entry, exit, dummy = self.Process.ATR_TR(self.Histories[ticker], indicator=indicator, buy_order=buy_order)
        df = self.clean(entry, exit, clean=True)
        profits = []
        for i, index in enumerate(df.index):
            if i%2 == 0:
                entry = self.Histories[ticker].Close[index]
            else:
                exit = self.Histories[ticker].Close[index]
                profit = exit - entry
                profits.append(profit)
        profits = np.array(profits)
        if np.isnan(profits.mean()):
            return 0, None
        return profits.mean(), profits
        
    def clean(self, entry, exit, clean=False):
        df = pd.concat([entry.to_frame(), exit.to_frame()], axis=1)
        df.index.name = 'Date/Time'
        df.sort_values('Date/Time', inplace=True)
        if clean:
            current=''
            for index in df.Entry.index:
                if np.isnan(df.Entry[index]):
                    if current == 'Entry':
                        df.drop(labels=index, axis=0, inplace=True)
                    else:
                        current = 'Entry'
                else:
                    if current == 'Exit':
                        df.drop(labels=index, axis=0, inplace=True)
                    else:
                        current = 'Exit'
        try:
            if np.isnan(df.Entry[df.index[0]]):
                df.drop(labels=df.index[0], axis=0, inplace=True)
            if np.isnan(df.Exit[df.index[-1]]):
                df.drop(labels=df.index[-1], axis=0, inplace=True)
        except IndexError:
            pass
        return df
    
    # following are the methods used to get the buying and sell times, and overall used in conjunction with
    # the Big_Money class in order to get exactly that
    
    def get_buy_sell(self):
        
        today = datetime.datetime.today()
        today = datetime.datetime(today.year, today.month, today.day)
        last_bday = today - BDay()
        today_is_bday = bool(len(pd.bdate_range(today, today)))
        
        df = pd.DataFrame(columns=['Buy', 'BuyTR', 'Sell', 'SellTR'])
        for stock, buy, sell in self.Goods:
            stuffs = self.Process.ATR_TR(self.Histories[stock], buy), self.Process.ATR_TR(self.Histories[stock], sell, buy_order=False)
            try:
                if today_is_bday:
                    df.loc[stock] = (stuffs[0][0].index[-1].to_pydatetime() == today), stuffs[0][2][-1], (stuffs[1][0].index[-1].to_pydatetime() == today), stuffs[1][2][-1]
                else:
                    df.loc[stock] = (stuffs[0][0].index[-1].to_pydatetime() == last_bday), stuffs[0][2][-1], (stuffs[1][0].index[-1].to_pydatetime() == last_bday), stuffs[1][2][-1]
            except IndexError:
                print('Nothing for '+stock)
        self.BuySellStop = df
        return self.BuySellStop
    
    def min_max_avg(self):
        parse = pd.Series(name='Calculator', dtype='float64')
        for ticker, dummy, dummy in self.Goods:
            parse[ticker] = self.Histories[ticker].Close[-1]
        return parse.min(), parse.max(), parse.mean()

In [320]:
stocks_master = pd.read_csv('ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt', sep='|')
stocks = stocks_master.Symbol[:-1]
stocks

0         AACG
1          AAL
2         AAME
3         AAOI
4         AAON
         ...  
3534     ZWZZT
3535    ZXYZ.A
3536     ZXZZT
3537      ZYNE
3538      ZYXI
Name: Symbol, Length: 3539, dtype: object

In [322]:
#stocks = ['aapl', 'msft', 'tsla', 'oxy', 'spy', 'amzn']
stonk = Stoinks(tickers=list(stocks[::100]), history=('2y', '1d'), margin=20)

AACG
AIKI
AOSL
AVEO
BLDR
BTAI
CEVA
CMLS
CSTE
DLTR
EIGR
EYES
FLXS
FXNC
GPAQ
HIBB
ICLR
IPKW
KIDS
LMB
MCHX
MRBK
NFLX
OCGN
PAACU
PIZ
PTI
REXN
SAVA
SLGL
SSNT
TCRR
TTEC
VCYT
WERN
ZBRA


In [310]:
stonk.get_buy_sell()

,Buy,BuyTR,Sell,SellTR
AIRTP,False,21.194456,False,19.402958
CDLX,False,88.794325,False,57.344055
CFMS,False,0.791881,False,2.330855
MTC,False,1.435319,False,17.662942
NFE,False,16.419532,False,16.645992
PROF,False,11.160235,False,14.468646
SOLO,False,0.947203,False,1.190971
AFIN,False,12.851305,False,7.152626
ASFI,False,11.870533,False,8.790661
ATXI,False,9.600144,False,10.687346


In [302]:
stonk.Profitable, stonk.Buyable, stonk.Sellable

({'AIRTP': {'buy_indicator': 'RSI', 'sell_indicator': 'SMA20'},
  'CDLX': {'buy_indicator': 'SMA50', 'sell_indicator': 'SMA50'},
  'CFMS': {'buy_indicator': 'HTITRENDLINE', 'sell_indicator': 'SMA50'},
  'MTC': {'buy_indicator': 'EMA12', 'sell_indicator': 'RSI'},
  'NFE': {'buy_indicator': 'SMA60', 'sell_indicator': 'MACD3'},
  'PHAT': {'buy_indicator': 'RSI', 'sell_indicator': 'SMA/EMA50'},
  'SYNA': {'buy_indicator': 'SMA50', 'sell_indicator': 'RSI'}},
 {'AFIN': {'buy_indicator': 'SMA60', 'sell_indicator': 'SMA50'},
  'ASFI': {'buy_indicator': 'TEMA', 'sell_indicator': 'SMA60'},
  'ATXI': {'buy_indicator': 'MACD1', 'sell_indicator': 'SMA/EMA50'},
  'BAND': {'buy_indicator': 'RSI', 'sell_indicator': 'SMA50'},
  'BVXV': {'buy_indicator': 'RSI', 'sell_indicator': 'SMA60'},
  'CODA': {'buy_indicator': 'SMA60', 'sell_indicator': 'SMA20'},
  'FLDM': {'buy_indicator': 'SMA/EMA50', 'sell_indicator': 'SMA/EMA50'},
  'OPCH': {'buy_indicator': 'RSI', 'sell_indicator': 'SMA50'},
  'TTEK': {'buy_i

In [305]:
stonk.Process.ATR_TR(stonk.Histories['DUO'], 'SMA20', buy_order=True)

(Series([], Name: Entry, dtype: float64),
 Series([], Name: Exit, dtype: float64),
 Series([], Name: TrailingPrice, dtype: float64))

In [311]:
x = [98, 6, 2]
y = [725, 48, 18]